In [1]:
#! pip3 install --upgrade pip
#! pip3 install pyspark
#! pip3 install pyspark-ai
#! pip3 install plotly
#! pip3 install openpyxl

In [2]:
import requests
import os
from io import BytesIO
from openpyxl import load_workbook
import pandas as pd
from pyspark_ai import SparkAI
from pyspark.sql import SparkSession

In [3]:
# Optional environment variables
os.environ["OPENAI_API_KEY"] = "..."
os.environ["JAVA_HOME"] ="..."

In [4]:
berkeley_db_version = "v8-May-2023"

In [5]:
berkeley_xls = f"https://gspp.berkeley.edu/assets/uploads/page/Voluntary-Registry-Offsets-Database--{berkeley_db_version}.xlsx"

In [ ]:
# Read the Berkeley database xls in a local workbook object
vcm_wb = load_workbook(filename=BytesIO(requests.get(berkeley_xls).content))

In [7]:
table_ws = vcm_wb['PROJECTS']

In [8]:
projects_csv = []
for row in table_ws.iter_rows(min_row=5):
    projects_csv.append({
        "Scope": row[5].value,
        "Type": row[6].value,
        "Country": row[10].value,
        "State": row[11].value
    })

In [9]:
df = pd.DataFrame.from_dict(projects_csv)

In [10]:
df.head(10)

,Scope,Type,Country,State
0,Household & Community,Bundled Energy Efficiency,Mali,Niono District
1,Chemical Processes,SF6 Replacement,United States,OREGON
2,Agriculture,Manure Methane Digester,United States,CALIFORNIA
3,Forestry & Land Use,Afforestation/Reforestation,Madagascar,Miandrivazo
4,Forestry & Land Use,Afforestation/Reforestation,Brazil,Roraima
5,Waste Management,Landfill Methane,United States,ILLINOIS
6,Industrial & Commercial,Fuel Switching,Bolivia,Nor Ciniti
7,Forestry & Land Use,Afforestation/Reforestation,United States,COLORADO
8,Industrial & Commercial,Pneumatic Retrofit,United States,OKLAHOMA
9,Waste Management,Landfill Methane,United States,ILLINOIS


In [ ]:
spark_ai = SparkAI()
spark_ai.activate()
spark = SparkSession.builder.appName("SparkAI x Berkeley VCM DB").getOrCreate()

In [12]:
spark_df = spark.createDataFrame(df)

In [40]:
spark_df.ai.plot("""
    A choropleth map visualising the number of projects in each country by project scope. 
    Show the different project scopes within each country as different series. 
    Ensure that the location mode is country names.
    The map area should be 800px.
    """)


INFO: Here is a Python code snippet that uses PySpark and Plotly to create a choropleth map visualizing the number of projects in each country by project scope. 


```
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px

# Start Spark session
spark = SparkSession.builder.appName('choropleth_map').getOrCreate()

# Assuming df is a Spark DataFrame
# Convert Spark DataFrame to Pandas DataFrame
pandas_df = df.toPandas()

# Group by Country and Scope, count the number of projects
grouped_df = pandas_df.groupby(['Country', 'Scope']).size().reset_index(name='Counts')

# Concatenate the dataframes
final_df = pd.concat([grouped_df])

# Create a choropleth map
fig = px.choropleth(final_df, 
                    locations="Country", 
                    color="Counts", 
                    hover_name="Country", 
                    animation_frame="Scope",
                    projection="natural earth",
                    height=800,
                    locationmo

23/08/17 08:53:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
